In [ ]:
import yaml
import pandas as pd

try:
    with open("../../config.yaml", "r") as file:
        config = yaml.safe_load(file)

    ticket_df = pd.read_csv(config['data']['sql']['ticket'], sep=",")
    schedule_df = pd.read_csv(config['data']['sql']['schedules'], sep=",")
    itinerary_df = pd.read_csv(config['data']['sql']['itinerary'], sep=",")

except:
    print("Yaml configuration file not found!")

In [ ]:
flight_df = pd.DataFrame(columns=["flight_id"])

In [ ]:
ticket_subset = ticket_df[["ticket_id", "airline_name_id", "flight_number_id"]]

# Concatenate with an empty flight_df (which will just give the indices aligned)
flight_df = pd.concat([ticket_subset], axis=1)

# Add flight_id as a sequential primary key
flight_df["flight_id"] = range(1, len(flight_df) + 1)

# Optional: reorder columns
flight_df = flight_df[["flight_id", "ticket_id", "airline_name_id", "flight_number_id"]]

flight_df

In [ ]:
flight_df.duplicated().any()

In [ ]:
flight_df.isnull().any()

In [ ]:
flight_df.to_csv(config['data']['sql']['flight'], index=False, encoding="utf-8")